### 학습 목표
- 결정 트리의 발전된 형태인 기초인 랜덤 포레스트를 학습시켜 중고차 가격을 예측.
- 교차 검증 방법을 활용하여 모델을 평가

### 학습 순서
- 문제 정의
- 라이브러리 및 데이터 불러오기 & 확인하기
- 전처리
  - 텍스트 데이터
  - 결측치 처리와 더미 변수 변환
- 모델링 및 예측하기
  - 이해하기 : K겹 교차검증, 랜덤 포레스트
- 하이퍼파라미터 튜닝

### 랜덤 포레스트 소개
- Random Forest 모델은 결정 트리의 단점인 오버피팅 문제를 완화시켜주는 발전된 형태의 트리 모델이다.
  - 랜덤으로 독립적인 트리를 여러 개 만들어서 결정 트리의 오버피팅 문제를 완화해 분류하는 알고리즘
- 랜덤으로 생성된 무수히 많은 트리를 이용하여 예측을 하기 때문에, 랜덤 포레스트라 불린다.
- 이렇게 여러 모델(여기서는 결정 트리)을 활용하여 하나의 모델을 이루는 기법은 "앙상블"이라 부른다
- 앙상블 기법 : 여러 모델을 만들고, 각 예측값들을 투표/평균 등으로 통합하여 더 정확한 예측을 도모하는 방법

### Top 10 선정 이유
- 앙상블 기법을 사용한 트리 기반 모델 중 가장 보편적인 방법
- 이후에 다루게 될 부스팅 모델에 비하면 예측력이나 속도에서 부족한 부분이 있고, 시각화에서는 결정 트리에 못미치나, 다음 단계인 부스팅 모델을 이해하려면 꼭 알아야 함

### 장점
- 결정 트리와 마찬가지로, 아웃라이어에 거의 영향을 받지 않는다.
- 선형/비선형 데이터에 상관없이 잘 작동한다.

### 단점
- 학습 속도가 상대적으로 느린 편이다.
- 수많은 트리를 동원하기 때문에 모델에 대한 해석이 어렵다

### 유용한 곳
- 종속변수가 연속형 데이터와 범주형 데이터인 경우 모두 사용할 수 있다
- 아웃라이어가 문제가 되는 경우 선형 모델보다 좋은 대안이 될 수 있다.
- 오버피팅 문제로 결정 트리를 사용하기 어려울 때, 랜덤 포레스틀를 사용할 수 있다.

### 9.1 문제 정의
- 알고리즘 : 랜덤 포레스트(Random Forest)
- 종속변수 : selling_price(판매 가격)
- 독립변수 : 생산년도, 주행거리, 변속기, 마일리지, 배기량 등
- 문제 유형 : 회귀
- 평가지표 : RMSE
- 사용한 모델 : RandomForestRegressor

## 9.2 라이브러리 및 데이터 불러오기, 데이터 확인하기

In [233]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

file_url = 'https://media.githubusercontent.com/media/musthave-ML10/data_source/main/car.csv'
data = pd.read_csv(file_url)

In [234]:
data.head()

,name,year,selling_price,km_driven,fuel,seller_type,transmission,owner,mileage,engine,max_power,torque,seats
0,Maruti Swift Dzire VDI,2014,450000,145500,Diesel,Individual,Manual,First Owner,23.4 kmpl,1248 CC,74 bhp,190Nm@ 2000rpm,5.0
1,Skoda Rapid 1.5 TDI Ambition,2014,370000,120000,Diesel,Individual,Manual,Second Owner,21.14 kmpl,1498 CC,103.52 bhp,250Nm@ 1500-2500rpm,5.0
2,Honda City 2017-2020 EXi,2006,158000,140000,Petrol,Individual,Manual,Third Owner,17.7 kmpl,1497 CC,78 bhp,"12.7@ 2,700(kgm@ rpm)",5.0
3,Hyundai i20 Sportz Diesel,2010,225000,127000,Diesel,Individual,Manual,First Owner,23.0 kmpl,1396 CC,90 bhp,22.4 kgm at 1750-2750rpm,5.0
4,Maruti Swift VXI BSIII,2007,130000,120000,Petrol,Individual,Manual,First Owner,16.1 kmpl,1298 CC,88.2 bhp,"11.5@ 4,500(kgm@ rpm)",5.0


In [235]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8128 entries, 0 to 8127
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   name           8128 non-null   object 
 1   year           8128 non-null   int64  
 2   selling_price  8128 non-null   int64  
 3   km_driven      8128 non-null   int64  
 4   fuel           8128 non-null   object 
 5   seller_type    8128 non-null   object 
 6   transmission   8128 non-null   object 
 7   owner          8128 non-null   object 
 8   mileage        7907 non-null   object 
 9   engine         7907 non-null   object 
 10  max_power      7913 non-null   object 
 11  torque         7906 non-null   object 
 12  seats          7907 non-null   float64
dtypes: float64(1), int64(3), object(9)
memory usage: 825.6+ KB


In [236]:
round(data.describe(), 2) # 통계 정보 출력

# 아웃라이어 존재
# => selling_price에서 max값이 유독 높다.
# => km_driven은 max, min 모두 아웃라이어
# 선형 모델은 아웃라이어에 대한 처리가 필요하지만, 트리 모델은 아웃라이어를 별로 처리하지 않는다.

,year,selling_price,km_driven,seats
count,8128.00,8128.00,8128.00,7907.00
mean,2013.80,638271.81,69819.51,5.42
std,4.04,806253.40,56550.55,0.96
min,1983.00,29999.00,1.00,2.00
25%,2011.00,254999.00,35000.00,5.00
50%,2015.00,450000.00,60000.00,5.00
75%,2017.00,675000.00,98000.00,5.00
max,2020.00,10000000.00,2360457.00,14.00


## 9.3 전처리 : 텍스트 데이터

In [237]:
# 문자형 데이터를 분리하는 데, 판다스 시리즈에서 제공하는 str.split()을 사용한다.
# => 판다스 데이터프레임이 아닌 시리즈에만 있는 함수이기 때문에, 컬럼 하나씩만 인덱싱해 처리해준다.
data['engine'].str.split(expand=True)

,0,1
0,1248,CC
1,1498,CC
2,1497,CC
3,1396,CC
4,1298,CC
...,...,...
8123,1197,CC
8124,1493,CC
8125,1248,CC
8126,1396,CC


## 9.3.1 engine 변수 전처리하기

In [238]:
data[['engine','engine_unit']]= data['engine'].str.split(expand=True)

In [239]:
data['engine'].head()

# 그러나, 결과물을 보면 맨 아래에 dtype: object로 되어 있다.
# => 즉, 숫자만 남았지만 이 데이터는 아직도 문자형 데이터로 인식되는 것

0    1248
1    1498
2    1497
3    1396
4    1298
Name: engine, dtype: object

In [240]:
# astype() 함수를 이용하여 숫자형 변수로 변환
data['engine'] = data['engine'].astype('float32')

In [241]:
# dtype이 float32로 변경됨
data['engine'].head()

0    1248.0
1    1498.0
2    1497.0
3    1396.0
4    1298.0
Name: engine, dtype: float32

In [242]:
# 문자 부분을 버리지 않고 굳이 남겨둔 이유는, 간혹 다른 단위를 가지는 데이터가 있을 수 있기 때문이다.
# => CC라는 값만 한 가지 존재한다
# => 따라서 해당 컬럼을 제거해도 된다.
data['engine_unit'].unique() # 고윳값 확인

array(['CC', nan], dtype=object)

In [243]:
data.drop('engine_unit', axis=1, inplace=True) # 변수 제거

## 9.3.2 max_power 변수 전처리하기

In [244]:
# 분할된 문자들을 새로운 변수들로 저장
data[['max_power', 'max_power_unit']] = data['max_power'].str.split(expand=True)

In [245]:
data['max_power'].head()

0        74
1    103.52
2        78
3        90
4      88.2
Name: max_power, dtype: object

In [246]:
# data['max_power'] = data['max_power'].astype('float32') # 숫자형 변수로 변환

# ValueError
# => 'bhp'라는 string을 float으로 바꿀 수 없다.

In [247]:
data[data['max_power'] == 'bhp'] # max_power 변수에 bhp라는 문자가 있는지 확인

# 데이터에 숫자 없이 bhp만 있어서, str.split()에서 빈 칸을 기준으로 분리할 때 분리되지 않은 채로 남은 것
# => 지금은 단 한 건의 데이터이기 때문에 간단히 라인을 제거하거나, 해당 값을 Null로 변환할 수 있다.
# => 하지만 Try and Except 블록을 활용할 것

,name,year,selling_price,km_driven,fuel,seller_type,transmission,owner,mileage,engine,max_power,torque,seats,max_power_unit
4933,Maruti Omni CNG,2000,80000,100000,CNG,Individual,Manual,Second Owner,10.9 km/kg,796.0,bhp,NaN,8.0,None


In [248]:
def isFloat(value): # 함수 정의
  try: 
    num = float(value)
    return num
  except ValueError:
    return np.NaN

In [249]:
# isFloat함수를 사용하여 숫자형 변수로 변환
data['max_power'] = data['max_power'].apply(isFloat)

In [250]:
# 고윳값 확인
# bhp이외에도 Null 값들이 있다. 적어도 다른 단위는 아니므로 따로 조치해줄 필요는 없다
data['max_power_unit'].unique()

array(['bhp', nan, None], dtype=object)

In [251]:
data.drop('max_power_unit', axis=1, inplace=True)

## 9.3.3 mileage 변수 전처리하기

In [252]:
# split() 함수로 분리된 숫자와 문자를 각각 별개의 컬럼에 두어야 하므로 expand 변수 사용
data[['mileage','mileage_unit']] = data['mileage'].str.split(expand=True)

In [253]:
data['mileage_unit'].unique()

# kmpl : km/1, 즉 리터당 킬로미터
# km/kg : 킬로그램당 킬로미터

# 킬로미터 : 주행거리
# 리터와 킬로그램 : 연로에 대한 측정 단위

array(['kmpl', 'km/kg', nan], dtype=object)

In [254]:
data['mileage'] = data['mileage'].astype('float32')

In [255]:
data['fuel'].unique()

array(['Diesel', 'Petrol', 'LPG', 'CNG'], dtype=object)

In [256]:
def mile(x):
    if x['fuel'] == 'Petrol':
        return x['mileage'] / 80.43
    elif x['fuel'] == 'Diesel':
        return x['mileage'] / 73.56
    elif x['fuel'] == 'LPG':
        return x['mileage'] / 40.85
    else: 
        return x['mileage'] / 44.23

In [257]:
data['mileage'] = data.apply(mile, axis=1)

In [258]:
data.drop('mileage_unit',axis=1,inplace=True)

## 9.3.4 torque 변수 전처리하기

In [259]:
data['torque'].head()

0              190Nm@ 2000rpm
1         250Nm@ 1500-2500rpm
2       12.7@ 2,700(kgm@ rpm)
3    22.4 kgm at 1750-2750rpm
4       11.5@ 4,500(kgm@ rpm)
Name: torque, dtype: object

In [260]:
data['torque'] = data['torque'].str.upper()

In [261]:
data['torque'].unique()

array(['190NM@ 2000RPM', '250NM@ 1500-2500RPM', '12.7@ 2,700(KGM@ RPM)',
       '22.4 KGM AT 1750-2750RPM', '11.5@ 4,500(KGM@ RPM)',
       '113.75NM@ 4000RPM', '7.8@ 4,500(KGM@ RPM)', '59NM@ 2500RPM',
       '170NM@ 1800-2400RPM', '160NM@ 2000RPM', '248NM@ 2250RPM',
       '78NM@ 4500RPM', nan, '84NM@ 3500RPM', '115NM@ 3500-3600RPM',
       '200NM@ 1750RPM', '62NM@ 3000RPM', '219.7NM@ 1500-2750RPM',
       '114NM@ 3500RPM', '115NM@ 4000RPM', '69NM@ 3500RPM',
       '172.5NM@ 1750RPM', '6.1KGM@ 3000RPM', '114.7NM@ 4000RPM',
       '60NM@ 3500RPM', '90NM@ 3500RPM', '151NM@ 4850RPM',
       '104NM@ 4000RPM', '320NM@ 1700-2700RPM', '250NM@ 1750-2500RPM',
       '145NM@ 4600RPM', '146NM@ 4800RPM', '343NM@ 1400-3400RPM',
       '200NM@ 1400-3400RPM', '200NM@ 1250-4000RPM',
       '400NM@ 2000-2500RPM', '138NM@ 4400RPM', '360NM@ 1200-3400RPM',
       '200NM@ 1200-3600RPM', '380NM@ 1750-2500RPM', '173NM@ 4000RPM',
       '400NM@ 1750-3000RPM', '400NM@ 1400-2800RPM',
       '200NM@ 1750-3000RP

In [262]:
def torque_unit(x):
    if 'NM' in str(x):
        return 'Nm'
    elif 'KGM' in str(x):
        return 'kgm'

In [263]:
data['torque_unit'] = data['torque'].apply(torque_unit)

In [264]:
data['torque_unit'].unique()

array(['Nm', 'kgm', None], dtype=object)

In [265]:
data['torque_unit'].isna()
data[data['torque_unit'].isna()]
# torque_unit이 결츶치인 라인의 torque 변수 고윳값 확인
data[data['torque_unit'].isna()]['torque'].unique()

array([nan, '250@ 1250-5000RPM', '510@ 1600-2400', '110(11.2)@ 4800',
       '210 / 1900'], dtype=object)

In [266]:
data['torque_unit'].fillna('Nm', inplace=True) # 결측치를 Nm으로 대체

In [267]:
def split_num(x):
  x = str(x)
  
  for i,j in enumerate(x):
    if j not in '0123456789.':
      cut = i
      break
  return x[:cut]

In [268]:
data['torque'] = data['torque'].apply(split_num)

In [269]:
data['torque']

0         190
1         250
2        12.7
3        22.4
4        11.5
        ...  
8123    113.7
8124       24
8125      190
8126      140
8127      140
Name: torque, Length: 8128, dtype: object

In [270]:
# data['torque'] = data['torque'].astype('float64')

In [271]:
data['torque'] = data['torque'].replace('', np.NaN)

In [272]:
data['torque'] = data['torque'].astype('float64')

In [273]:
data['torque'].head()

0    190.0
1    250.0
2     12.7
3     22.4
4     11.5
Name: torque, dtype: float64

In [274]:
def torque_trans(x):
    if x['torque_unit'] == 'kgm':
        return x['torque'] * 9.8066
    else:
        return x['torque']

In [275]:
data['torque'] = data.apply(torque_trans, axis=1)

In [276]:
data.drop('torque_unit', axis=1, inplace=True)

In [277]:
data.head()

,name,year,selling_price,km_driven,fuel,seller_type,transmission,owner,mileage,engine,max_power,torque,seats
0,Maruti Swift Dzire VDI,2014,450000,145500,Diesel,Individual,Manual,First Owner,0.318108,1248.0,74.00,190.00000,5.0
1,Skoda Rapid 1.5 TDI Ambition,2014,370000,120000,Diesel,Individual,Manual,Second Owner,0.287384,1498.0,103.52,250.00000,5.0
2,Honda City 2017-2020 EXi,2006,158000,140000,Petrol,Individual,Manual,Third Owner,0.220067,1497.0,78.00,124.54382,5.0
3,Hyundai i20 Sportz Diesel,2010,225000,127000,Diesel,Individual,Manual,First Owner,0.312670,1396.0,90.00,219.66784,5.0
4,Maruti Swift VXI BSIII,2007,130000,120000,Petrol,Individual,Manual,First Owner,0.200174,1298.0,88.20,112.77590,5.0


## 9.3.5 name 변수 처리하기

In [278]:
data['name'] = data['name'].str.split(expand=True)[0]
# name 변수를 공백으로 나누고 첫 번째 부분으로 업데이트

In [279]:
data['name'].unique()

array(['Maruti', 'Skoda', 'Honda', 'Hyundai', 'Toyota', 'Ford', 'Renault',
       'Mahindra', 'Tata', 'Chevrolet', 'Fiat', 'Datsun', 'Jeep',
       'Mercedes-Benz', 'Mitsubishi', 'Audi', 'Volkswagen', 'BMW',
       'Nissan', 'Lexus', 'Jaguar', 'Land', 'MG', 'Volvo', 'Daewoo',
       'Kia', 'Force', 'Ambassador', 'Ashok', 'Isuzu', 'Opel', 'Peugeot'],
      dtype=object)

In [280]:
data['name'] = data['name'].replace('Land', 'Land Rover')

## 9.4 전처리: 결측치 처리와 더미 변수 변환

In [281]:
data.isna().mean() # 변수별 결측치의 평균 확인

# mileage, engine, seats의 경우 결측치 비율이 동일한 것으로 보아 특정 행에서 milieage 이하 컬럼이 전혀 수집되지 않은 것으로 보임.

name             0.000000
year             0.000000
selling_price    0.000000
km_driven        0.000000
fuel             0.000000
seller_type      0.000000
transmission     0.000000
owner            0.000000
mileage          0.027190
engine           0.027190
max_power        0.026575
torque           0.027313
seats            0.027190
dtype: float64

In [282]:
data.dropna(inplace=True) # 결측치 행 제거
len(data)

7906

In [283]:
# 결측치가 제거되었으므로, 남은 텍스트 컬럼을 더미 변수로 변환
data = pd.get_dummies(data, columns = ['name','fuel','seller_type','transmission','owner'], drop_first=True)

In [284]:
data.head()

,year,selling_price,km_driven,mileage,engine,max_power,torque,seats,name_Ashok,name_Audi,...,fuel_Diesel,fuel_LPG,fuel_Petrol,seller_type_Individual,seller_type_Trustmark Dealer,transmission_Manual,owner_Fourth & Above Owner,owner_Second Owner,owner_Test Drive Car,owner_Third Owner
0,2014,450000,145500,0.318108,1248.0,74.00,190.00000,5.0,0,0,...,1,0,0,1,0,1,0,0,0,0
1,2014,370000,120000,0.287384,1498.0,103.52,250.00000,5.0,0,0,...,1,0,0,1,0,1,0,1,0,0
2,2006,158000,140000,0.220067,1497.0,78.00,124.54382,5.0,0,0,...,0,0,1,1,0,1,0,0,0,1
3,2010,225000,127000,0.312670,1396.0,90.00,219.66784,5.0,0,0,...,1,0,0,1,0,1,0,0,0,0
4,2007,130000,120000,0.200174,1298.0,88.20,112.77590,5.0,0,0,...,0,0,1,1,0,1,0,0,0,0


## 9.5 모델링 및 평가하기

In [285]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data.drop('selling_price', axis=1), data['selling_price'], test_size=0.2, random_state=100)

In [286]:
from sklearn.ensemble import RandomForestRegressor

In [288]:
model = RandomForestRegressor(random_state=100)
model.fit(X_train, y_train)
train_pred = model.predict(X_train)
test_pred = model.predict(X_test)

In [289]:
from sklearn.metrics import mean_squared_error
print("train_rmse", mean_squared_error(y_train, train_pred) ** 0.5,
"test_rmse", mean_squared_error(y_test, test_pred) ** 0.5) # rmse 확인

train_rmse 53531.41548125947 test_rmse 131855.18391308116


## 9.6 이해하기: K-폴드 교차검증 (K-fold Cross-Validation)
- 교차검증(cross validation) 의 목적은 모델의 예측력을 더 안정적으로 평가하기 위함 (교차 타당성 이라고도 한다.)
- 교차검증 : 다양한 훈련셋/시험셋을 통하여 모델에 더 신뢰할 수 있는 평가를 하는 방법

- K-fold Cross-Validation의 아이디어
  - 데이터를 특정 개수(K개)로 쪼개어서 그중 하나씩을 선택하여 시험셋으로 사용하되, 이 과정을 K번 반복하는 것

In [290]:
from sklearn.model_selection import KFold
data

# 인덱스는 0부터 8127까지 있지만, 실제로는 7906줄이다.
# => 중간에 dropna()를 사용하여 약 2%정도 되는 줄을 제거했기 때문
# => KFold는 이 인덱스 값을 이용하여 데이터를 분할하는데, 이와 같이 중간에 빈 값이 존재하면 에러가 발생
# => reset_index()를 사용하여 인덱스를 정리

,year,selling_price,km_driven,mileage,engine,max_power,torque,seats,name_Ashok,name_Audi,...,fuel_Diesel,fuel_LPG,fuel_Petrol,seller_type_Individual,seller_type_Trustmark Dealer,transmission_Manual,owner_Fourth & Above Owner,owner_Second Owner,owner_Test Drive Car,owner_Third Owner
0,2014,450000,145500,0.318108,1248.0,74.00,190.00000,5.0,0,0,...,1,0,0,1,0,1,0,0,0,0
1,2014,370000,120000,0.287384,1498.0,103.52,250.00000,5.0,0,0,...,1,0,0,1,0,1,0,1,0,0
2,2006,158000,140000,0.220067,1497.0,78.00,124.54382,5.0,0,0,...,0,0,1,1,0,1,0,0,0,1
3,2010,225000,127000,0.312670,1396.0,90.00,219.66784,5.0,0,0,...,1,0,0,1,0,1,0,0,0,0
4,2007,130000,120000,0.200174,1298.0,88.20,112.77590,5.0,0,0,...,0,0,1,1,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8123,2013,320000,110000,0.230014,1197.0,82.85,113.70000,5.0,0,0,...,0,0,1,1,0,1,0,0,0,0
8124,2007,135000,119000,0.228385,1493.0,110.00,235.35840,5.0,0,0,...,1,0,0,1,0,1,1,0,0,0
8125,2009,382000,120000,0.262371,1248.0,73.90,190.00000,5.0,0,0,...,1,0,0,1,0,1,0,0,0,0
8126,2013,290000,25000,0.320419,1396.0,70.00,140.00000,5.0,0,0,...,1,0,0,1,0,1,0,0,0,0


In [291]:
data.reset_index(drop=True, inplace=True) # 인덱스를 변수로 가져옴

In [293]:
data

# drop 매개변수를 사용하지 않으면 기존 인덱스가 새로운 컬럼 형태로 추가되기 때문에, drop=True를 꼭 입력해준다

,year,selling_price,km_driven,mileage,engine,max_power,torque,seats,name_Ashok,name_Audi,...,fuel_Diesel,fuel_LPG,fuel_Petrol,seller_type_Individual,seller_type_Trustmark Dealer,transmission_Manual,owner_Fourth & Above Owner,owner_Second Owner,owner_Test Drive Car,owner_Third Owner
0,2014,450000,145500,0.318108,1248.0,74.00,190.00000,5.0,0,0,...,1,0,0,1,0,1,0,0,0,0
1,2014,370000,120000,0.287384,1498.0,103.52,250.00000,5.0,0,0,...,1,0,0,1,0,1,0,1,0,0
2,2006,158000,140000,0.220067,1497.0,78.00,124.54382,5.0,0,0,...,0,0,1,1,0,1,0,0,0,1
3,2010,225000,127000,0.312670,1396.0,90.00,219.66784,5.0,0,0,...,1,0,0,1,0,1,0,0,0,0
4,2007,130000,120000,0.200174,1298.0,88.20,112.77590,5.0,0,0,...,0,0,1,1,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7901,2013,320000,110000,0.230014,1197.0,82.85,113.70000,5.0,0,0,...,0,0,1,1,0,1,0,0,0,0
7902,2007,135000,119000,0.228385,1493.0,110.00,235.35840,5.0,0,0,...,1,0,0,1,0,1,1,0,0,0
7903,2009,382000,120000,0.262371,1248.0,73.90,190.00000,5.0,0,0,...,1,0,0,1,0,1,0,0,0,0
7904,2013,290000,25000,0.320419,1396.0,70.00,140.00000,5.0,0,0,...,1,0,0,1,0,1,0,0,0,0


In [294]:
kf = KFold(n_splits=5) # 5개로 분할

In [296]:
X = data.drop('selling_price', axis=1) # 종속변수 제거하여 X에 저장
y = data['selling_price']

In [299]:
# 훈련셋과 시험셋으로 분리
# 순차적으로 이터레이션 1 ~ 이터레이션 5에 사용될 인덱스
# 왼쪽 리스트는 훈련셋에 사용될 인덱스, 오른쪽 리스트는 시험셋에 사용될 인덱스
for i, j in kf.split(X):
  print(i, j)

[1582 1583 1584 ... 7903 7904 7905] [   0    1    2 ... 1579 1580 1581]
[   0    1    2 ... 7903 7904 7905] [1582 1583 1584 ... 3160 3161 3162]
[   0    1    2 ... 7903 7904 7905] [3163 3164 3165 ... 4741 4742 4743]
[   0    1    2 ... 7903 7904 7905] [4744 4745 4746 ... 6322 6323 6324]
[   0    1    2 ... 6322 6323 6324] [6325 6326 6327 ... 7903 7904 7905]


In [300]:
for train_index, test_index in kf.split(X):
  X_train, X_test = X.loc[train_index], X.loc[test_index]
  y_train, y_test = y[train_index], y[test_index] # y는 시리즈 형태이므로 loc를 사용하지 않고 곧바로 인덱싱 가능
  

In [301]:
train_rmse_total = []
test_rmse_total = []

In [302]:
for train_index, test_index in kf.split(X):
  X_train, X_test = X.loc[train_index], X.loc[test_index]
  y_train, y_test = y[train_index], y[test_index]

  model = RandomForestRegressor(random_state=100)
  model.fit(X_train, y_train)
  train_pred = model.predict(X_train)
  test_pred = model.predict(X_test)

  train_rmse = mean_squared_error(y_train, train_pred) ** 0.5
  test_rmse = mean_squared_error(y_test, test_pred) ** 0.5

  train_rmse_total.append(train_rmse)
  test_rmse_total.append(test_rmse)

In [303]:
train_rmse_total

[50825.5556350298,
 58854.04054344074,
 57904.19615940739,
 56218.23740006373,
 58967.150857632456]

In [305]:
print("train_rmse: ", sum(train_rmse_total) / 5, "test_rmse: ", sum(test_rmse_total) / 5)

# 교차검증을 사용하기 전과 약간의 차이
# => RMSE가 더 올라간 , 조금 더 정확한 평가 결과

train_rmse:  56553.836119114814 test_rmse:  142936.58918244042


## 9.7 이해하기: 랜덤 포레스트
- 랜덤 포레스트 : 결정 트리의 집합체
- 결정 트리를 사용하면 오버피팅 문제가 쉽게 발생하는데, 랜덤 포레스트는 여러 트리를 활용하여 최종 모델을 만든다. 즉 다양한 트리의 의견을 반영하기 때문에 오버피팅 위험을 낮출 수 있다.
- 랜덤 포레스트가 여러 개의 트리를 만들 때는 데이터 전체를 사용하지 않고, 매번 다른 일부의 데이터를 사용하여 다른 트리를 만들어낸다.
  - 당연히 일부 데이터만 사용하면, 전체 데이터를 사용한 결과보다 예측값이 떨어지겠지만, 예측력이 다소 떨어지는 수많은 트리들이 함께 모여 중윗값을 찾아내면 오버피팅을 막는 데 효율적이다.

## 9.8 하이퍼파라미터 튜닝

In [306]:
train_rmse_total = []
test_rmse_total = []

for train_index , test_index in kf.split(X):
    X_train , X_test = X.iloc[train_index,:],X.iloc[test_index,:]
    y_train , y_test = y[train_index] , y[test_index]
    model = RandomForestRegressor(n_estimators = 300, max_depth = 50, min_samples_split = 5, min_samples_leaf = 1, n_jobs=-1, random_state=100)
    model.fit(X_train,y_train)
    train_pred = model.predict(X_train)
    test_pred = model.predict(X_test)
     
    train_rmse = mean_squared_error(y_train, train_pred) ** 0.5
    test_rmse = mean_squared_error(y_test, test_pred) ** 0.5
    
    train_rmse_total.append(train_rmse) 
    test_rmse_total.append(test_rmse) 

In [308]:
print("train_rmse:", sum(train_rmse_total)/5, "test_rmse:",sum(test_rmse_total)/5)

# train_rmse는 하이퍼파라미터 튜닝 전보다 다소 높아졌으나, test_rmse는 조금 더 낮아졌다.
# => 조금이나마 오버피팅이 줄어드니, 새로운 데이터를 예측하기에는 더 좋은 모델

# K폴드 교차검증을 사용하면 경우에 따라 더 높은 RMSE가 나올 수 있다.
# => K폴드 교차검증은 더 낮은 RMSE를 얻기 위함이 아니라, 우연의 요소가 배제된 더 신뢰할 만할 결과를 얻기 위함

train_rmse: 66762.84568886801 test_rmse: 142205.83441414658
